In [1]:
import os
from openai import OpenAI

In [2]:
# client = OpenAI(
#     api_key = os.environ['OPENAI_API_KEY'] The constructor now fetches api_key by itsels.
# )

client = OpenAI()

# chat completion


In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content
    

### Response structure

```
ChatCompletion(
    id='chatcmpl-9eNYu4oeZ4UHW5mrlIiAR7DETWmzF',
    choices=[
        Choice(
        finish_reason='stop', index=0, logprobs=None,
        message=ChatCompletionMessage(
        content='OUTPUT_MESSAGE',
        role='assistant', function_call=None, tool_calls=None))],
        created=1719410720,
        model='gpt-3.5-turbo-0125',
        object='chat.completion',
        service_tier=None,
        system_fingerprint=None,
        usage=CompletionUsage(completion_tokens=222, prompt_tokens=21, total_tokens=243)
)
```

In [4]:
get_completion("what is 1+1")

'1+1 equals 2.'

# Introduction to Langchain chat APIs and LangChain Expression Language (LCEL)

- langchain_openai.ChatOpenAI
- langchain.chat_models.ChatOpenAI
- langchain_core.messages.(HumanMessage, SystemMessage)
- langchain_core.output_parsers.StrOutputParser
- langchain_core.prompts.ChatPromptTemplate

## Using models directly

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

### Creating model 

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following text from English into Italian"),
    HumanMessage(content="I am Bharatiya."),
]

result = model.invoke(messages) # String output

### Parsing output

In [26]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

**Using parser seperately**

In [27]:
parser.invoke(result)

'Sono Bharatiya.'

### using "chain"

" | " this is chain operator, it takes output of the previous operation and pass it as input to next operation.

In [28]:
chain = model | parser

This chain has two steps: first the language model is called, then the result of that is passed to the output parser

In [30]:
chain.invoke(messages)

'Io sono Bharatiya.'

## Prompt Templates

prompt templates takes the raw user input, applies transformations such as adding system message and formatting the user input according to a template, and return a prompt that is ready to pass into a language model.

In [31]:
from langchain_core.prompts import ChatPromptTemplate

In [38]:
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")] # we can chose the variable name.
)

In [39]:
# Have to use same variable names used during creation of prompt_template
result = prompt_template.invoke({"language": "italian", "text":"My nationality is Bharatiya"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='My nationality is Bharatiya')])

### Chaining prompt_template, model and parser

In [40]:
chain = prompt_template | model | parser

In [42]:
chain.invoke({"language":"italian", "text": "My nationality is Bharatiya"})

'La mia nazionalità è Bharatiya.'